<a href="https://colab.research.google.com/github/Renatobcb/ML_DL/blob/main/ml_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Um modelo de Sistema de Alerta Precoce (EWS) para crises financeiras
"""

import numpy as np
import hashlib

class Config:
    """ Creates a config object that specifies how the data is processed and how the experiment is run.
        The default values assigned here can be altered by the user in the experiment files (see experiments folder)
     """
    

    def __init__(self):

         # Path of R is needed as the decision tree is trained in R.
        self.r_path = None # e.g. 'C:\\Program Files\\R\\R-3.5.1\\bin\\x64\\Rscript'
        
        #### The following parameters determine how the data is processed ####
        self.data_predictors = ["drate", "cpi_pdiff" , "bmon_gdp_rdiff", "stock_pdiff",
                                 "cons_pdiff" ,
                                 "pdebt_gdp_rdiff", "inv_gdp_rdiff", "ca_gdp_rdiff",
                                 "tloan_gdp_rdiff",
                                 "tdbtserv_gdp_rdiff", "global_loan"]  # Names of the indicators used as predictors
        # pdiff: percentage change
        # rdiff ratio change. Change of the variable relative to the change in GDP
        self.data_horizon = 2  # Horizon of percentage and ratio changes (in years)

        self.data_period = 'all'  # The time frame investigate. Either 'all' observations,
        # or 'pre-ww2' or 'post-ww2'.
        self.data_exclude_extreme_period = True  # Whether to exclude WW1, WW2 and 
        # the Great Depression
        self.data_include_crisis_year = False  # Whether to exclude the actual crisis
        # observation and only predict years a head of a crisis
        self.data_years_pre_crisis = 2  # number of years before a crisis for
        # which outcome is set positive
        self.data_post_crisis = 4  #  How many observations (in years) after the 
        # crisis should be deleted to avoid post-crisis bias
        
        
        
        #### The following parameters determine experimental details ####

        self.exp_n_kernels = 1  # The number of kernels of the CPU used in parallel
        self.exp_nfolds = 5  # Number of folds in the cross-validation experiment.

        self.exp_algos = ['extree', "log"]  # list of algorithms that are tested in the experiment

        self.exp_year_split = None  # If 'None' the cross-validation experiment is run.
        # If it is a year y all instances up to that year are used for training and 
        # the following observations for testing the model. The latter option is used for forecasting

        self.exp_id = "crisis"
        # This variable specifies constraints for the cross-validation
        # 'no': no constraint used
        # 'crisis': the observation of a crisis (by default 1-2 years before crisis)
        #   are assigned to the same fold
        # 'year': all observations of a certain year are assigned to the same fold
        # 'year_and_crisis' combination of the two constraints above

        # Hyperparameter search
        self.exp_verbose = 0  # Determines how verbose the output of the hyperparameter search is. 
        self.exp_hyper_folds = 5  # Number of folds in the cross-validation of the hyperparameters
        self.exp_rep_cv = 1  # How often the cross-validation of the hyperparameters is repeated.
        self.exp_search = "grid"  # Either we use full 'grid' search or 'random' search
        self.exp_n_iter_rsearch = 250  # How many hyperparamter combinations are tested in the random search
        self.exp_optimization_metric = 'roc_auc'  # Metric that is optimized in the hyperparameter search

        # Shapley
        self.exp_do_shapley = True  # Whether Shapley values are computed
        self.exp_shap_background = 50  # Number of background samples used by the Shapley Kernel explainer
        self.exp_shapley_interaction = False  # Whether interactions of Shapley values are computed

        self.exp_error_costs = "0.5"  # cost associated with the false positive 
        # and false negative error. If set to '0.5', both errors are treated as equally important
        #  If set to 'balanced' the error of the minority classes is upweighted 
        # such that the product of the error-weight and the proportion of objects
        # in the class is equivalent for both classes.

        # The error costs can also be set to arbitrary values using a dictionary,
        # e.g. {0: 0.1, 1: 0.9}. This means that the error in the positive class
        # are 9 times more important than the error in the negative class.

        self.exp_do_upsample = False  # whether the minority class is upsampled 
        # according to the error costs (see above). If False, the objects are weighted
        # according to the error costs. Note that the weighting of objects is not
        # supported by all algorithms.

        self.exp_bootstrap = "no" # bootstrapping the training set with the options
        # no (no bootstrappoing), up (upsampling), down (downsampling)
        self.exp_bootstrap_replace = "no" # # whether to resample the minority class by replacement as well


    def _make_name(self, name_appx=""):
        """Creates a descriptive name according to the configuration.
        This name is used when saving the files in the results folder.
        It is based on some of the experiments parameters but the user
        can also add a suffix to the name with the name_appx argument
        """
        name_data = name_appx +  str(self.data_horizon) + "_" + str(self.exp_id)

        if self.exp_year_split is None:
            expName = "CV"
        else:
            expName = "year" + str(int(self.exp_year_split))
        name = self.data_period + "_" + str(expName) + "_" + str(name_data)

        if self.data_include_crisis_year:
            name = name + "crsIncl_"

        if self.exp_do_shapley:
            name = name + "SHAP_"


        return name

In [ ]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 6.6 MB/s eta 0:00:00


In [ ]:
"""
This script provides utility functions that are used in the experiments
"""

import sys
import warnings
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as st
import shap
import os
from operator import itemgetter
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import mstats
import statsmodels.formula.api as smf


def shapley_kernel_wrapper(model, trainx, testx, config):
    """ This function is called by the prediction algorithms (ml_functions) 
    to compute the Shapley values. Note that the decision tree based models
    such as random forest do provide faster and exact (non-approximated)
     Shapley values with the TreeShapExplainer"""
    if config.exp_shap_background >= trainx.shape[0]:
        background = trainx
    else:
        background = shap.kmeans(trainx, config.exp_shap_background)

        # random sample of background values
        # ixx = np.random.choice(trainx.shape[0], config.exp_shap_background, replace=False)
        # background = trainx[ixx, :]
        # print(background.shape)

        explainer = shap.KernelExplainer(model.predict_proba, background)
        if isinstance(model, LogisticRegression):  # one background instance is enough if we use a linear model
            background = shap.kmeans(trainx, 1)
            backup_base = explainer.fnull
            explainer = shap.KernelExplainer(model.predict_proba, background)
            explainer.fnull = backup_base

    fnull_save = explainer.fnull[1]
    out = [explainer.shap_values(testx[i, :], l1_reg=0.0)[1] for i in np.arange(len(testx))]
    return np.vstack(out)

def exclude_periods(data, config):
    """ the function sets all cue values on the excluded periods to NA and returns
     an index of all the objects that should later be deleted.
     This way of processing ist best because all the preprocessing functions do not need
     consider cases where years are already missing """

    exclude_ix = np.zeros(len(data)) > 1

    if config.data_exclude_extreme_period:
        # exclude great depression | but NOT the beginnig of this crisis
        exclude_ix = exclude_ix | (np.array(data["year"] > 1933) & np.array(data["year"] < 1939))
        # exclude WW1
        exclude_ix = exclude_ix | (np.array(data["year"] > 1913) & np.array(data["year"] < 1919))
        # exclude WW2
        exclude_ix = exclude_ix | (np.array(data["year"] > 1938) & np.array(data["year"] < 1946))

    if not config.data_period in ['all', 'pre-ww2', 'post-ww2']:
        raise ValueError("time split is either 'all', 'pre-ww2', or 'post-ww2'")

    elif config.data_period  == 'pre-ww2':
        exclude_ix = exclude_ix | np.array(data["year"] > 1939)

    elif config.data_period == 'post-ww2':
        exclude_ix = exclude_ix | np.array(data["year"] < 1946)

    feature_names = set(data.columns.values).difference(set(['year', 'country', 'iso', 'crisis_id',
                                                             'crisis']))
    # set all feature values to NA in the excluded periods
    data.loc[exclude_ix, feature_names] = np.nan

    return data, exclude_ix




def create_grouped_folds(y, y_group, y_group_2=None, nfolds=10, reps=1, balance=True):
    """Create folds such that all objects in the same y_group and in the same
    y_group_2 (if not none) are assigned
    to the same fold
    :param np.array y: Binary outcome variable
    :param np.array y_group: Grouping variable, e.g crisis indicator
    :param np.array y: Second grouping variable (optional)
    :param int nfolds: Number of folds
    :param int reps: Number of replications of the n-fold cross-validation
    :param bool balance: If true, the outcome y is balanced as much as possible,
        i.e. that there are an equal number of
        positive observations in each fold
    """
    no = y.size
    iterable = list()
    for r in np.arange(reps):
        placed = np.zeros(no, dtype=np.int64)
        out = np.zeros(no, dtype=np.int64)*np.nan
        pos_counter = np.zeros(nfolds, dtype=np.int64)
        neg_counter = np.zeros(nfolds, dtype=np.int64)

        # go through objects in random order
        oo = np.random.choice(np.arange(no), no, replace=False)
        for i in oo:
            if placed[i] == 0:
                if not y_group_2 is None: # no verlap in year AND crisis_id
                    ix = np.where((y_group[i] == y_group) | (y_group_2[i] == y_group_2))[0]
                    for i in np.arange(25):
                        ix = np.where(np.in1d(y_group, y_group[ix]) | np.in1d(y_group_2, y_group_2[ix]))[0]
                else: # no overlap in crisis_id
                    ix = np.where(y_group[i] == y_group)[0]

                placed[ix] = 1

                if balance:
                    if y[i] == 1:
                        rf = np.random.choice(np.where(pos_counter == pos_counter.min())[0])
                        pos_counter[rf] += ix.size
                    else:
                        rf = np.random.choice(np.where(neg_counter == neg_counter.min())[0])
                        neg_counter[rf] += ix.size
                else:
                    rf = int(np.random.randint(0, nfolds, 1))

                out[ix] = rf

        for f in np.arange(nfolds):
            ix_train = np.where(out != f)[0]
            ix_test = np.where(out == f)[0]
            # make sure that test set contains both classes
            if (not (y[ix_test].mean() == 0)) & (not (y[ix_test].mean() == 1)):
                iterable.append((ix_train, ix_test))

    if len(iterable) < nfolds*reps:
        print("Repeat folding, some test set had zero variance in criterion")
        return create_grouped_folds(y, y_group, y_group_2=y_group_2,
                                  nfolds=nfolds, reps=reps, balance=balance)
    else:
        return iterable, out

def create_forecasting_folds(y, year, min_crisis=20, temp_resolution=1):
    """ Create folds for the forecasting experiment
     :param np.array y: Binary outcome variable
     :param np.array year: Time stamp for each observation
     :param int min_crisis: Minimum number of crisis observations in the training set.
     :param int temp_resolution: After how many years a new model should be trained.
     The default is 1, meaning, a new model is trained for every year.
    """
    iterable = list()
    last_train_year = list()
    uni_years = sorted(year.unique())
    del uni_years[-1]
    for i in np.arange(len(uni_years)):
        n_crisis = y[year <= uni_years[i]].sum()
        if (n_crisis >= min_crisis) & ((uni_years[i] % temp_resolution) == 0):
            ix_train = np.where(year <= uni_years[i-1])[0]
            ix_test = np.where(year > uni_years[i - 1])[0]

            if (len(ix_train) > 0) & (len(ix_test) > 0):
                iterable.append((ix_train, ix_test))
                last_train_year.append(uni_years[i])
    return iterable, last_train_year


def hyperparam_search(model, parameters, use='grid', n_jobs=1, cv=None, scoring=None,
                  n_iter=250, verbose=False):
    """Create a Grid or random search object that can be processed by Scikit-learn"""
    if isinstance(cv, int):
        raise ValueError("The argument cv should not be a number because the GridSearch algorithms"
                         " in sklearn do always create the same folds even with differnt random seeds."
                         " Rather you should pass folds that were created by our own function create_grouped_folds")


    if np.cumprod([len(parameters[x]) for x in parameters.keys()]).max() <= n_iter: # do use gridsearch if less than n_iter
        use = "grid" # combinations are tested
    if use == 'grid':
        model_out = GridSearchCV(model, parameters, n_jobs=n_jobs, cv=cv,
                                 scoring=scoring, verbose=verbose, iid = True)
    if use == 'random':
        model_out = RandomizedSearchCV(model, parameters, n_jobs=n_jobs, 
                                       cv=cv, n_iter=n_iter, scoring=scoring,

                                       verbose=verbose)
    return model_out


def write_file(data, file_name, path = '../results/', round=3, format=".txt",
              short_name=6, append=False, shorten = True):
      
    """ Writes a table as a text file to the hard drive """
    out = data.round(round)
    if not os.path.exists(path):
        os.mkdir(path)
    if isinstance(data, pd.core.frame.DataFrame):
        if shorten:
            out.columns = [x.replace("_" , "")[0:short_name] for x in out.columns.values]
            out.index = [str(x).replace('_', ' ')[0:short_name] for x in out.index.values]
        if append:
            out.to_csv(path + file_name + format , sep='\t', mode = 'a', header=True)
        else:
            out.to_csv(path + file_name + format, sep='\t', header=True)


def weights_from_costs(costs, Y):
    """  Weights observations according to the costs of the errors (as speceificed by the user) of the two classes.
    For example if the cost vector is [0.5, 0.5] and class A is twice as prevalent as class B,
    objects in class B will get twice the weight as objects in class A. """
    p1 = Y.mean()
    weights = {}
    weights[1] = costs[1] / (p1 * costs[1] + (1-p1) * costs[0])
    weights[0] = costs[0] / (p1 * costs[1] + (1 - p1) * costs[0])
    return weights


def downsample(X, Y, costs={0:0.5, 1: 0.5}, group=None):
    """ downsample the majority class according to the costs of the errors. """
    if group is None:
        group = np.arange(len(Y))
    weights = weights_from_costs(costs, Y)

    ix_pos = np.where(Y == 1)[0]
    n_pos = ix_pos.size
    ix_neg = np.where(Y == 0)[0]
    n_neg = ix_neg.size
    norm_w = min(weights.values())
    weights[0] = weights[0] / norm_w
    weights[1] = weights[1] / norm_w

    if weights[0] > weights[1]:
      ix_pos = np.random.choice(ix_pos, size=int(round(n_pos/weights[0])), replace=True)
    else:
      ix_neg = np.random.choice(ix_neg, size=int(round(n_neg/weights[1])), replace=True)
    ixses = np.concatenate((ix_pos, ix_neg))
    ixses = np.random.choice(ixses, size=ixses.size, replace=False)
    return X[ixses, :], Y[ixses], group[ixses]

def upsample(X, Y, group, costs):
    """ upsamples the minority class """
    weights = weights_from_costs(costs, Y)

    ix_pos = np.where(Y == 1)[0]
    n_pos = ix_pos.size
    ix_neg = np.where(Y == 0)[0]
    n_neg = ix_neg.size
    norm_w = min(weights.values())
    weights[0] = weights[0] / norm_w
    weights[1] = weights[1] / norm_w

    if weights[1] > weights[0]:
      ix_pos = np.random.choice(ix_pos, size=int(round(weights[1] * n_pos)), replace=True)
    else:
      ix_neg = np.random.choice(ix_neg, size=int(round(weights[0] * n_neg)), replace=True)
    ixses = np.concatenate((ix_pos, ix_neg))
    ixses = np.random.choice(ixses, size=ixses.size, replace=False)

    return X[ixses, :], Y[ixses], group[ixses]


# UTILITIES FOR TRANSFORMING VARIABLES #

def make_ratio(data_input, variables, denominator="gdp"):
    """ Computes the ratio of two variables. By detault the denominator is GDP. """

    names_out = []
    if isinstance(variables, str):
        variables = [variables]
    data = data_input.copy()
    for var in variables:
        varname = var + '_' + denominator
        data[varname] = data[var] / data[denominator]
        names_out.append(varname)
    return data, names_out

def make_shift(data_input, variables, type, horizon=5):
    """ Computes the change of a variable with respect to a certain horizon.
     :param pd.dDtaFrame data_input: Dataset. The tranformed variable will be appended to that data
     :param list of str variables : Name of the variables in data_input that will be transformed
     :param str type: Type of transformation. Either "absolute" (change) or "percentage" (change).
    """
    
    names_out = []
    data = data_input.copy()
    data_group = data.groupby('iso')
    if isinstance(variables, str):
        variables = [variables ]
    for var in variables:
        if type == "absolute":
            varname = var + '_rdiff' + str(horizon)
            data[varname] = data_group[var].diff(horizon)
        elif type == "percentage":
            varname = var + '_pdiff' + str(horizon)
            # attention objects must be ordered by year and country as they are in the original data
            data[varname] = data_group[var].apply(lambda x: lag_pct_change(x, h=horizon))
            #data[varname] = data_group[var].pct_change(horizon)

        names_out.append(varname)
    return data, names_out

def lag_pct_change(x, h):
    """ Computes percentage changes """
    lag = np.array(pd.Series(x).shift(h))
    return (x - lag) / lag



def make_level_change(data_input, variables, type, horizon=10):
    """ Computes the hamilton filter or difference from moving average
     :param pd.dDtaFrame data_input: Dataset. The tranformed variable will be appended to that data
     :param list of str variables: Name of the variables in data_input that will be transformed
     :param str type: Type of transformation. Either "ham" (hamilton filter) or "mad" (movgin average difference).
    """
    names_out = []
    data = data_input.copy()
    data_grouped = data.groupby('iso')
    if isinstance(variables, str):
        variables = [variables]
    for var in variables:
        if type == "mad":
            varname = var + '_mad'
            data[varname] = np.nan
            data_mad = pd.DataFrame(data_grouped.apply(mov_ave_diff, var, horizon), 
                                    columns=[varname])
            for iso in data_mad.index.values:
                data.loc[data.iso == iso, varname] = data_mad.loc[iso, varname]

        if type == "ham":
            varname = var + '_ham'
            data[varname] = np.nan
            data_ham = pd.DataFrame(data_grouped.apply(hamilton_filter, var, 2, 4),
                                    columns=[varname])
            for iso in data_ham.index.values:
                data.loc[data.iso == iso, varname] = data_ham.loc[iso, varname]
        names_out.append(varname)
    return data, names_out


def make_relative_change(data_input, variables, index='gdp', horizon=5):
    """ Computes the change of a variable relative the the change of of another variable
     :param pd.dDtaFrame data_input: Dataset. The tranformed variable will be appended to that data
     :param list of str variables: Name of the variables in data_input that will be transformed
     :param str index: Name of the variables to which the change is relative to (default is GDP)
    """
    names_out = []
    data = data_input.copy()
    data_grouped = data.groupby('iso')
    if isinstance(variables, str):
        variables = [variables]
    for var in variables:
        varname = var + '_idiff' + str(horizon)
        data[varname] = np.nan
        data_idiff = pd.DataFrame(data_grouped.apply(index_ratio_change, var,
                                                     index, horizon), columns=[varname])
        for iso in data_idiff.index.values:
            data.loc[data.iso == iso, varname] = data_idiff.loc[iso, varname]
        names_out.append(varname)
    return data, names_out


def mov_ave_diff(group, col, L=10):
    """ Computes the gap between a moving average (of length L) and the
    observations on a grouped data set """
    values = group[col].values
    N = len(values)
    out = np.zeros(N) * np.nan
    if N >= L:
        for i in range(N - L + 1):
            out[i + L - 1] = values[i + L - 1] - np.mean(values[i-1:i + L-1])
    return out

def index_ratio_change(group, ind1, ind2, l=5):
    """relative change of ind1 to ind2 over period l for group values."""

    val1 = group[ind1].values
    val2 = group[ind2].values
    N = len(val1)
    out = np.zeros(N) * np.nan

    if N >= l:
        for i in range(N - l):
            out[i + l] = (val1[i + l]  / val1[i]) / (val2[i + l] / val2[i]) - 1
    return out


def hamilton_filter(group, col, h=2, p=4, output="cycle"):  
    """ computes Hamilton filter
    : param int h: look-head period
    : param int p: number of lagged variables
    """

    x = group[col].values
    # note: Hamilton used 100 times x's logrithm in his employment data,
    # however, this is commented out because our data has negative values
    # x = 100*np.log(x)
    # Get the trend/predicted series
    trend = hamilton_filter_glm(x, h, p)
    if trend is not None:  # if dataframe passed is not full of nans
        # Get the cycle which is simple the original series substracted by the trend
        cycle = x - trend
        # Get the random walk series which is simply the difference between
        # the original series and the h look back
        df_x = pd.DataFrame(x)
        df_x_h = df_x.shift(h)
        random = df_x - df_x_h
    else:
        trend = x
        cycle = x
        random = x
    # Return required series in result, if all is selected then all results
    # are returned in a data frame
    if (output == "x"):
        return x
    elif (output == "trend"):
        return trend
    elif (output == "cycle"):
        return np.asarray(cycle)
    elif (output == "random"):
        return random
    elif (output == "all"):
        df = pd.DataFrame()
        df['x'] = x
        df['trend'] = trend
        df['cycle'] = cycle
        df['random'] = random
        df.plot()
        # pyplot.show()
        return df
    else:
        print ('\nInvalid output type')



def hamilton_filter_glm(x, h=2, p=4):
    """ Runs the linear model for the specification of the hamilton filter """
    # Create dataframe for time series to be smoothed, the independent variable y
    df = pd.DataFrame(x)
    df.columns = ['yt8']
    # Create matrix of dependent variables X which are the shifts of 8 period back
    # for 4 consecutive lags on current time t
    for lag in range(h, (h + p)):
        df['xt_' + str(lag - h + 1)] = df.yt8.shift(lag)
    # Getting the dependent varaibles X's index names
    X_columns = []
    for i in range(1, p + 1):
        new_s = 'xt_' + str(i)
        X_columns.append(new_s)
    # y and X variables for regression
    y = df['yt8']
    X = df[X_columns]

    xt_0 = pd.DataFrame(np.ones((df.shape[0], 1)))
    xt_0.columns = ['xt_0']
    X = xt_0.join(X)
    # Build the OLS regression model and drop the NaN
    try:
        if (sum(np.isnan(y)) != y.size):
            model = sm.OLS(y, X, missing='drop').fit()
            # Optional: print out the statistics
            model.summary()
            predictions = model.predict(X)
            return predictions
        else:
            return y
    except ValueError:
        pass

def all_same(items):
    return all(x==items[0] for x in items)

def sigmoid(x):
  return 1/(1 + np.exp(-x))

def sigmoidinv(x):
  return -np.log(1.0/x -1)

def normalize(data):
  return data.apply(normalizeV)

def normalizeV(x):
  x = x.astype(dtype="float32")
  return (x- np.nanmin(x))/(np.nanmax(x) - np.nanmin(x))

def performance_results(Y_in, Y_pred_in, threshold = 0.5):
    """ Computes performance metrics
    : param np.array Y_in: true values (0 or 1) of the response variable
    : param np.array Y_pred_in: predicted values of the response variable (between 0 an 1)
    : param float threshold: if Y_pred_in >= threshold, the predcited class is positive, otherwise negative
    """
    Y_pred_in = np.array(Y_pred_in, dtype=float)
    # types of Y and Y_pred are pd.Seres
    ix_miss = np.isnan(Y_pred_in)

    Y = Y_in[~ix_miss].copy()
    Y_pred = Y_pred_in[~ix_miss].copy()

    n = Y.size
    Y = Y * 1 # typecast boolean variables
    Y_pred = Y_pred * 1  # typecast boolean variables

    Y_bool = np.array(Y,dtype = "bool")
    Y_pred_bool = Y_pred >= threshold
    # True positive (tp), ture negative (tn), false positive (fp), false negative (fn)
    tp = np.logical_and(Y_bool, Y_pred_bool).sum()
    tn = np.logical_and(~Y_bool, ~Y_pred_bool).sum()
    fp = np.logical_and(~Y_bool, Y_pred_bool).sum()
    fn = np.logical_and(Y_bool, ~Y_pred_bool).sum()

    out = dict()
    if any(pd.isna(Y_pred)):
        return dict([
            ("accuracy", float("nan")),
            ("balanced", float("nan")),
            ("tp_rate", float("nan")),
            ("fp_rate", float("nan")),
            ("auc", float("nan")),
            ("tp", float("nan")),
            ("tn", float("nan")),
            ("fp", float("nan")),
            ("fn", float("nan")),
        ])
    else:
        out['tp'] = tp
        out['tn'] = tn
        out['fp'] = fp
        out['fn'] = fn
        out['accuracy'] = float(tp + tn) / float(n)
        if ((tp + fn == 0)):
            out['tp_rate'] = float("nan")
        else:
            out['tp_rate'] = float(tp) / float(tp + fn)

        if (tn + fp == 0):
            out['fp_rate'] = float("nan")
        else:
            out['fp_rate'] = 1 - float(tn) / float(tn + fp)

        out['balanced'] = (out["tp_rate"] + (1 - out["fp_rate"])) / 2.0

        if ((tp + fn > 0) & (tn + fp > 0)):
            out['auc'] = roc_auc_score(Y_bool, Y_pred)
        else:
            out['auc'] = float('nan')
        return out


def remove_file(file_path):
    """ removes file from hard drive """
    if os.path.exists(file_path):
        os.remove(file_path)


In [ ]:
"""
This script loads the Jorda-Schularick-Taylor Macrohistry database and transforms the variables.
"""
import pandas as pd
import numpy as np
import re

def create_data(config):
    """ Create the data set from the raw data from "http://www.macrohistory.net/data/"
    according to the specifications in the Config object"""
    
    # the data can be downlaoded directly:
    # df_jst = pd.read_excel('http://www.macrohistory.net/JST/JSTdatasetR3.xlsx', sheet_name="Data") 
    df_jst = pd.read_excel('data/JSTdatasetR3.xlsx', sheet_name="Data") 
    df = df_jst.copy()
    # rename variables
    df.rename(columns={
        "crisisJST": "crisis",
        'stir': 'srate',
        'ltrate': 'lrate',
        'iy': 'inv_gdp',
        'debtgdp': 'pdebt_gdp',
        'money': 'bmon',
        'narrowm': 'nmon',
        'tloans': 'tloan',
        'tbus': 'bloan',
        'thh': 'hloan',
        'tmort': 'mort',
        'stocks': 'stock',
        'hpnom': 'hp',
        'rconpc': 'cons'
    }, inplace=True)

    horizon = config.data_horizon
    predictors = config.data_predictors
    # we do not compute growth rates for the interest rates and the slope for the yield curve.
    no_change = ["drate",  "global_drate", "lrate", "srate"]

    # For the other predictors we compute growth rate (percentage change or ratio change)
    # and add the horizon (e.g. 2 year change) to the variable name
    predictors = [p + str(horizon) for p in predictors if p not in no_change] +\
        list(set(predictors).intersection(set(no_change)))

    df, exclude_ix = exclude_periods(df, config)  # exclude periods that are not normal economic conditions (e.g. WW2)

    df.loc[:, 'drate'] = df['lrate'] - df['srate']  # rate differential

    df.loc[:, 'pdebt'] = df['pdebt_gdp'] * df['gdp'] # compute public debt from public debt/gdp ratio
    df.loc[:, 'inv'] = df['inv_gdp'] * df['gdp']  # compute investment from investment/gdp ratio

    # Calculaute debt to service ratios
    df.loc[:, 'tdbtserv'] = df['tloan'] * df['lrate'] / 100.0
    
    pre_gdp_ratios = ['bmon', 'nmon', 'tloan', 'bloan',
                      'hloan', 'mort', 'ca', 'cpi', 'tdbtserv',
                      'inv', 'pdebt'] # vector of variables that will be transformed by GDP ratio
    df, gdp_ratios = make_ratio(df, pre_gdp_ratios, denominator='gdp')

    # here we compute the transformations and att the variables to the dataset df 

    # ratio change by GDP (rdiff)
    df, _ = make_shift(df, ["lrate", "srate", "drate"] + gdp_ratios,
                       type="absolute", horizon=horizon)
    # percentage change (pdiff)
    df, _ = make_shift(df, ['stock', 'cpi', 'hp', 'cons', 'gdp'] + pre_gdp_ratios,
                       type="percentage",
                       horizon=horizon)  # do not use absolute change
    # hamilton filter (ham)
    df, _ = make_level_change(df, ["cons"] + gdp_ratios, type="ham")


    # --- Computing global variables --- #

    # global credit growth (global_loan)
    for year in df["year"].unique():
        ix = df["year"] == year
        for country in df["iso"].unique():
            # computing the average across all countries but the selected one
            perc_pos = df.loc[ix.values & (df.iso != country).values,
                              "tloan_gdp_rdiff" + str(horizon)].mean()

            if not np.isnan(perc_pos):
                df.loc[ix.values & (df.iso == country).values,
                       "global_loan" + str(horizon)] = perc_pos

    # global slope of the yield curve
    for year in df["year"].unique():
        ix = df["year"] == year
        for country in df["iso"].unique():
            # computing the average across all countries but the selected one
            perc_pos = df.loc[ix.values & (df.iso != country).values, "drate"].mean()

            if not np.isnan(perc_pos):
                df.loc[ix.values & (df.iso == country).values, "global_drate"] = perc_pos

    # check whether we have created all features that will be used in the experiment
    if len(set(predictors).difference(set(df.columns.values))) > 0:
        raise ValueError('Features ' + ', '.join(set(predictors).difference(set(df.columns.values))) + "\n" +
                         "could not be found in the data!")




    # --- creating the 'landing zone' on the crisis outcome --- #
    years = df.year.values
    isos = df['iso'].values

    crisis_in = df_jst.crisisJST.values == 1
    crisis = crisis_in * 0
    for i, (yr, cr) in enumerate(zip(years, crisis_in)):
        if cr:
            for l in np.arange(1, 1 + config.data_years_pre_crisis): # flagging years before crisis as positive 
                if yr > (np.min(years) + l - 1):
                    crisis[i - l] = 1
            if config.data_include_crisis_year: 
                crisis[i] = 1  # crisis year

    # --- treatment of actual crisis and post crisis observations --- #
    i_keep = np.ones(len(df), dtype=int)
    for i, (yr, cr, iso) in enumerate(zip(years, crisis_in, df.iso)):
        if cr:
            if not config.data_include_crisis_year: 
                i_keep[i] = 0

            for j in range(1, 1 + config.data_post_crisis):
                k = i + j
                if (iso == df.iso[k]) & (k < len(df)):
                    i_keep[k] = 0

    # --- Give all observations of the same crisis the same ID --- #
    # This ID is used for cross-validation to make sure that the same crisis
    # is not in the training and test set
    # This function generalizes to any length of crises

    crisis_id = np.zeros(len(df))
    count = int(1)
    for i in np.arange(2, len(df)):
        if crisis[i] == 1:
            if not ((crisis[i - 1] == 1) & (isos[i] == isos[i - 1])):
                count += 1
            crisis_id[i] = count
    # All other observations get unique identifier
    crisis_id[crisis_id == 0] = np.random.choice(sum(crisis_id == 0),
                                                 size=sum(crisis_id == 0),
                                                  replace=False) + 2 + int(max(crisis_id))

    # create the data set
    features = df.loc[:, predictors]
    data = features
    data['crisis'] = crisis.astype(int)
    data['crisis_id'] = crisis_id.astype(int)
    data['year'] = years.astype(int)
    data['iso'] = isos # name of countries

    exclude_ix = exclude_ix | (i_keep == 0)
    data = data.loc[~exclude_ix, :]

    data = data.dropna()  # remove missing values
    data = data.reset_index(drop=True)  # update index
    return data

In [ ]:
"""
This script contains wrapper functions for the different machine learning methods.
"""
import os
import shap
import subprocess
import time

from sklearn import svm as sk_svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn import tree
from sklearn.utils import resample
from sklearn.base import BaseEstimator, ClassifierMixin


# Hyperparameter space for the support vector machines
svm_cspace = 2. ** np.linspace(-5., 10., 10)
svm_gammaspace = 2. ** np.linspace(-10., 3., 10)


class PredictionModel:

    """ This class is used to train all models, compute the Shapley values, and 
        summarise the output in a dictionary
    """
    def __init__(self, model, name, data, config, **kwargs):
        """
         :param object model: Prediction model object in the standard sklearn format.
         :param str name: name given to the model.
         :param dict data: contains the training and test data
         :param Config config: config object. 
        """
        self.trainx = data["trainx"]
        self.trainy = data["trainy"]
        self.testx = data["testx"]
        self.config = config
        self.model = model
        self.name = name
        start_time = time.time()
        self._train() # train model 
        stop_time = time.time()

        self.best_hyper = None
        if hasattr(self.model, "best_params_"):
            self.best_hyper = model.best_params_
        if hasattr(self.model, "best_estimator_"):
            self.model = self.model.best_estimator_
        
        self.shapV, self.shapV_inter = self._compute_shap() # compute Shapley values

        self.output = {
            "name": name,
            "pred": model.predict_proba(self.testx)[:, 1],
            "fit": model.predict_proba(self.trainx)[:, 1],
            "model": self.model,
            "hyper_params": self.best_hyper,
            "shapley": self.shapV,
            "shapley_inter": self.shapV_inter,
            "time": stop_time - start_time
        }

    def _train(self, **kwargs): # train the prediction model and obtain preditions
        self.model.fit(self.trainx, self.trainy, **kwargs)
    
    def _compute_shap(self): # compute Shapley values
        shapV_inter = None # Shapley values of the interaction of variables
        shapV = None # Shapley values of the individual variables

        if self.config.exp_do_shapley:
        
            if self.name in ["extree", "forest"]: # TreeExplainer
                explainerTree = shap.TreeExplainer(self.model)
                shapV = explainerTree.shap_values(self.testx)[1]
            
                if self.config.exp_shapley_interaction: # compute Shapley interaction
                    shapV_inter = explainerTree.shap_interaction_values(self.testx)[1]
                
            else: # KernelExplainer
                shapV = shapley_kernel_wrapper(self.model, self.trainx,
                                               self.testx, self.config)
   
        return (shapV, shapV_inter)

def gaussianprocess(data, config, name, **kwargs):
    
    model = GaussianProcessClassifier()
    model_instance = PredictionModel(model, name, data, config, **kwargs)
    return model_instance.output


def logreg(data, config, sample_weight, name, **kwargs):
    # Logistic regression
    
    model = LogisticRegression(penalty="none", solver = "lbfgs")
    model_instance = PredictionModel(model, name, data, config,
                                     sample_weight = sample_weight)
    return model_instance.output



def extree(data, config, sample_weight, cv_hyper, do_cv, name, **kwargs):
    # Extremely randomised trees.
    # We use the default parameters (do_cv = False) in the paper, as hyperparameter tuning does not improve the performance
    
    if do_cv:
        hyperparameters = {'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                           'max_depth': [2, 3, 4, 5, 7, 10, 12, 15, 20]
                           }
        model = hyperparam_search(ExtraTreesClassifier(n_estimators=1000,  n_jobs=1),
                                    hyperparameters,
                                    use=config.exp_search,
                                    n_jobs=config.exp_n_kernels, cv=cv_hyper,
                                    scoring=config.exp_optimization_metric,
                                    n_iter=config.exp_n_iter_rsearch,
                                    verbose=config.exp_verbose)
    else:
        
        model = ExtraTreesClassifier(n_estimators=1000, n_jobs=config.exp_n_kernels)

    model_instance = PredictionModel(model, name, data, config,
                                     sample_weight = sample_weight)
    return model_instance.output

def forest(data, config, sample_weight, cv_hyper, do_cv, name, **kwargs):
    # Random forest.
    # We use the default parameters (do_cv = False) in the paper, as hyperparameter tuning does not imporve the performance

    if do_cv:
       
        hyperparameters = {'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                           'max_depth': [2, 3, 4, 5, 7, 10, 12, 15, 20]
                           }
        model = hyperparam_search(RandomForestClassifier(n_estimators=1000,  n_jobs=1),
                                    hyperparameters,
                                    use=config.exp_search,
                                    n_jobs=config.exp_n_kernels, 
                                    cv=cv_hyper,
                                    scoring=config.exp_optimization_metric,
                                    n_iter=config.exp_n_iter_rsearch,
                                    verbose=config.exp_verbose)

    else:
        
        model = RandomForestClassifier(n_estimators=1000, n_jobs=config.exp_n_kernels)

    model_instance = PredictionModel(model, name, data, config, sample_weight = sample_weight)
    return model_instance.output

def nnet_single(data, cv_hyper, config, name, **kwargs):
    # Single neural network
    
    n_features = data["trainx"].shape[1]
    hyperparameters = {'alpha': 10.0 ** np.linspace(-3.0, 3.0, 10),
                       'hidden_layer_sizes': list(
                               set([round(n_features / 3.0), round(n_features / 2.0), n_features,
                                    (n_features, n_features),
                                    (n_features, round(n_features / 2.0)),
                                    (n_features*2, n_features), 
                                    (n_features*2, n_features*2)
                                    ])),
                        'activation': ['tanh', 'relu']}
    
    # Exclude single neuron or zero neuron network
    hyperparameters["hidden_layer_sizes"] = list(set(hyperparameters["hidden_layer_sizes"]).difference(set([0, (1, 0)])))
    
    model = hyperparam_search(MLPClassifier(solver='lbfgs'),
                               hyperparameters,
                               use=config.exp_search,
                               n_jobs=config.exp_n_kernels, cv=cv_hyper,
                               scoring=config.exp_optimization_metric,
                               n_iter=config.exp_n_iter_rsearch,
                               verbose=config.exp_verbose)
    model_instance = PredictionModel(model, name, data, config)
    return model_instance.output



def nnet_multi(data, config, group, name,  **kwargs):
    # Neural network ensemble
    ''' Fitting this ensemble is very slow and only recommended on a high performance cluster. The ensemble #
    searchers for hyperparameters for each of the 25 base model in the ensemble to increase the variance
     across models '''
    
    resample="bootstrap" # resample is one of the following ["bootstrap", "copy", "upsample"], 
    start = time.time()
    n_features = data["trainx"].shape[1]
    hyperparameters = {'alpha': 10.0 ** np.linspace(-3.0, 3.0, 10),
                       'hidden_layer_sizes': list(
                               set([round(n_features / 3.0), round(n_features / 2.0), n_features,
                                    (n_features, n_features),
                                    (n_features, round(n_features / 2.0)),
                                    (n_features*2, n_features), 
                                    (n_features*2, n_features*2)
                                    ])),
                        'activation': ['tanh', 'relu']}
    
    # Exclude single neuron or zero neuron network
    hyperparameters["hidden_layer_sizes"] = list(set(hyperparameters["hidden_layer_sizes"]).difference(set([0, (1, 0)])))
    
    model = NnetMultiObj(resample=resample, config=config,
                     hyperparameters=hyperparameters, group=group)
    model_instance = PredictionModel(model, name, data, config)
    return model_instance.output
    
class NnetMultiObj(BaseEstimator, ClassifierMixin):
    # Train neural networks in the neural network ensemble  
    
    start = time.time()

    def __init__(self, resample, config, hyperparameters, group):
        self.models = list()
        self.n_models = 25
        self.resample = resample
        self.hyperparameters = hyperparameters
        self.config = config
        self.group = group

    def fit(self, X, y=None):
        for _ in np.arange(self.n_models):

            if self.resample == "bootstrap":
                x_rs, y_rs, group_rs = resample(X, y, self.group, replace=True)
            elif self.resample == "upsample":
                x_rs, y_rs, group_rs = upsample(X, y,
                                                group=self.group,
                                                costs={0: y.mean(), 1: 1 - y.mean()})
            else: x_rs, y_rs, group_rs = X, y, self.group

            cv_hyper, cv_fold_vector = create_grouped_folds(y_rs, group_rs, nfolds=5, reps=1)

            m = hyperparam_search(MLPClassifier(solver='lbfgs'),
                                  self.hyperparameters,
                                  use=self.config.exp_search,
                                  n_jobs=self.config.exp_n_kernels, cv=cv_hyper,
                                  scoring=self.config.exp_optimization_metric,
                                  n_iter=self.config.exp_n_iter_rsearch,
                                  verbose=self.config.exp_verbose)
            m.fit(x_rs, y_rs)
            self.models.append(m)
        return self

    def predict_proba(self, X, y=None):
        predm = np.zeros((X.shape[0], self.n_models, 2)) * np.nan
        for m in np.arange(len(self.models)):
            predm[:, m, :] = self.models[m].predict_proba(X)
        return predm.mean(axis=1)


def svm_single(data, cv_hyper, config, sample_weight, name, **kwargs):
    # Support-vector machine with radial basis function kernel
    
    hyperparameters= {'C': svm_cspace, 'gamma': svm_gammaspace}
    model = hyperparam_search(sk_svm.SVC(kernel='rbf', probability=True),
                          hyperparameters,
                          use=config.exp_search,
                          n_jobs=config.exp_n_kernels,
                          cv=cv_hyper,
                          scoring=config.exp_optimization_metric,
                          n_iter=config.exp_n_iter_rsearch,
                          verbose=config.exp_verbose)
    model_instance = PredictionModel(model, name, data, config,
                                     sample_weight = sample_weight)
    return model_instance.output



def svm_multi(data, config, group, sample_weight, name, **kwargs):
    # Support vector machine ensemble ensemble
    ''' Fitting this ensemble is very slow and only recommended on a high performance cluster. The ensemble #
    searchers for hyperparameters for each of the 25 base model in the ensemble to increase the variance
     across models '''
      

    resample = "upsample" # resample is one of the following ["none", "bootstrap", "copy", "upsample"]

    if config.exp_do_upsample and (resample=="upsample"):
       raise ValueError("The SVM ensemble upsamples the data already, It is not recommended to upsample another time \
           using the the exp_do_upsample of the Config class.")
       
    hyperparameters = {'C': svm_cspace, "gamma": svm_gammaspace}
    model = SvmMultiObj(config=config, hyperparameters=hyperparameters,
                        group=group, resample=resample,
                       sample_weight=sample_weight)
    
    model_instance = PredictionModel(model, name, data, config)
    return model_instance.output
  
class SvmMultiObj(BaseEstimator, ClassifierMixin):
       # Train support vector machines in the support vector machine ensemble  
    start = time.time()
    def __init__(self, config, hyperparameters, group, resample, sample_weight):
        self.models = list()
        self.n_models = 25 # number of models in the ensemble
        self.hyperparameters = hyperparameters
        self.config = config
        self.group = group
        self.resample = resample
        self.sample_weight = sample_weight

    def fit(self, X, y=None):
        for _ in np.arange(self.n_models):

            if self.resample == "bootstrap":
                x_rs, y_rs, group_rs = resample(X, y, self.group, replace=True)
            elif self.resample == "upsample":
                x_rs, y_rs, group_rs = upsample(X, y, 
                                                group=self.group,
                                                costs={0: y.mean(), 1: 1 - y.mean()})
            else: x_rs, y_rs, group_rs = X, y, self.group


            cv_hyper, cv_fold_vector = create_grouped_folds(y_rs, group_rs, nfolds=5, reps=1)

            m = hyperparam_search(sk_svm.SVC(kernel='rbf', probability=True),
                              self.hyperparameters,
                              use=self.config.exp_search,
                              n_jobs=self.config.exp_n_kernels, cv=cv_hyper,
                              scoring=self.config.exp_optimization_metric,
                              n_iter=self.config.exp_n_iter_rsearch,
                              verbose=self.config.exp_verbose)
            if self.resample == "upsample":
                m.fit(x_rs, y_rs)
            else:
                m.fit(x_rs, y_rs, sample_weight=self.sample_weight)

            self.models.append(m)
        return self

    def predict_proba(self, X, y=None):
        predm = np.zeros((X.shape[0], self.n_models, 2)) * np.nan
        for m in np.arange(len(self.models)):
            predm[:, m, :] = self.models[m].predict_proba(X)
        return predm.mean(axis=1)

def rmodel(name, costs, config, ident, size):
    # Wrapper function to train models in R.
    # The actual models in R are trained in the r_run.R script
   
    if config.r_path is None:
        raise ValueError("""Please specify the path of R using the Config attribute r_path.
     For example, in the experiment script, add the line:
     config.r_path = 'C:\\Program Files\\R\\R-3.5.1\\bin\\x64\\Rscript.""")
    
    start = time.time()
    rfile = 'r_data/out_to_python_' + str(name) + str(ident) + '.csv'
    FNULL = open("r_data/r_log.txt", 'w')  # suppress output
    
    subprocess.check_call([config.r_path, 'scripts\\r_run.R',
                           str(costs[1]), str(costs[0]), str(ident),
                           str(config.exp_n_kernels), str(size), name],
                          shell=False, stdout=FNULL, stderr=subprocess.STDOUT)
       
    while True:    
        try:
            pred_r = pd.read_csv(rfile, index_col=0)["x"].values
            break
        except:
            pass
    out = {"name": name,
           "pred": pred_r,
           "fit": None,
           "model": None,
           "hyper_params": None,
           "shapley": None,
           "shapley_inter": None,
           "time": (time.time() - start)
           }
    return out

In [ ]:
"""
This script contains the functions that conduct the cross-validation and forecasting experiments.
"""

import sys
import random
import xarray as xr
import time
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_sample_weight



def train_and_test(df, config):

    """ Low level experiment function
    It samples the training and test data, trains and test the prediction models 
    (their function are in ml_functions.py)
    either with our without the computation of Shapley values
    
     :param pd.DataFrame df: Data set with an arbitrary number of predictors and the columns
        crisis, crisis_id, year, and iso.
     :param Config config: Configuration file that specifies the experimental setup
    """

    algo_names = config.exp_algos
    nfolds = config.exp_nfolds
    hyper_folds = config.exp_hyper_folds
    rep_cv = config.exp_rep_cv

    X = df.copy()
    Y = X['crisis'].values.astype(int)
    crisis_id = X['crisis_id'].values.astype(int)
    years = X.year.values

    X = X.drop(columns=['crisis', 'crisis_id', 'year', 'iso'])
    feature_names = X.columns.values
    X = np.array(X)

    output_ypred = pd.DataFrame(index=np.arange(len(X)), columns=algo_names)

    # prepare a list that contains all the results
    model_out = {x: list() for x in algo_names}
    fits_out = {x: list() for x in algo_names}
    time_out = {x: list() for x in algo_names}
    params_out = {x: list() for x in algo_names}

    if config.exp_do_shapley:
        output_shapley = xr.DataArray(np.zeros((len(algo_names),
                                                X.shape[0],
                                                X.shape[1])) * float("nan"),
                                      [('algorithms', algo_names),
                                       ("instances", np.arange(X.shape[0])),
                                       ("features", feature_names)])
        output_shapley_fnull = pd.DataFrame(index=np.arange(len(X)),
                                            columns=algo_names)
    else:
        output_shapley = None
        output_shapley_fnull = None

    if config.exp_shapley_interaction:
        inter_algos = list(set(algo_names).intersection(set(["extree", "forest"])))
        output_shapley_inter = xr.DataArray(np.zeros((len(inter_algos),
                                                X.shape[0],
                                                X.shape[1], X.shape[1])) * float("nan"),
                                      [('algorithms', inter_algos),
                                       ("instances", np.arange(X.shape[0])),
                                       ("features1", feature_names),
                                       ("features2", feature_names)])
    else:
        output_shapley_inter = None

    results = {'predictions': output_ypred,
               "fits": fits_out,
               'ix_test': [],
               'ix_train': [], 
               'models': model_out,
               'parameters': params_out, "time": time_out,
               'shapley': output_shapley,
               "shapley_fnull": output_shapley_fnull,
               "data": [],
               'shapley_inter': output_shapley_inter}


    if config.exp_year_split is None:
        # Create the cross-validation folds
        if (config.exp_id == "none"):
            folds, _ = create_grouped_folds(y=Y, y_group=np.arange(Y.shape[0]),
                                          nfolds=nfolds, reps=1)

        if(config.exp_id == "crisis"):
            folds, _ = create_grouped_folds(y=Y, y_group=crisis_id, nfolds=nfolds,
                                          reps=1)

        if (config.exp_id == "year_and_crisis"):
            folds, _ = create_grouped_folds(y=Y, y_group=crisis_id, y_group_2=years,
                                          nfolds=nfolds, reps=1)
        if (config.exp_id == "year"):
            folds, _ = create_grouped_folds(y=Y, y_group=years, nfolds=nfolds,
                                          reps=1)


    else:
        # If we have a year splitting training and test set:
        nfolds = 1

    # run through the folds
    for f in np.arange(nfolds):
        sys.stdout.write('.')

        if config.exp_year_split is None:
            # obtain training and test set from the previously defined folds
            ix_train = list(folds[f][0])
            ix_test = list(folds[f][1])
        else:
            # observations before splitting year are used for training, 
            # the remaining observations for testing
            ix_train = list(np.where(years <= config.exp_year_split)[0])
            ix_test = list(np.where(years > config.exp_year_split)[0])

        # a random shuffle of the order of observations.
        ix_train = np.array(random.sample(ix_train, len(ix_train)))
        ix_test = np.array(random.sample(ix_test, len(ix_test)))

        if config.exp_bootstrap == "naive":
            ix_train = np.random.choice(ix_train, size=len(ix_train), replace=True)

        if config.exp_bootstrap in ["up", "down"]:
            ix_pos = ix_train[Y[ix_train] == 1]
            ix_neg = ix_train[Y[ix_train] == 0]
            replacer = False
            if config.exp_bootstrap_replace == "yes":
                replacer = True # whether to sample the minoritty class by replacement as well

            if config.exp_bootstrap == "up":
                if len(ix_neg) > len(ix_pos):
                    ix_train = np.concatenate((np.random.choice(ix_neg,
                                                                size=len(ix_neg), 
                                                                replace=replacer),
                                               np.random.choice(ix_pos,
                                                                size=len(ix_neg),
                                                                replace=True)))

                if len(ix_pos) > len(ix_neg):
                    ix_train = np.concatenate((np.random.choice(ix_pos,
                                                                size=len(ix_pos),
                                                                replace=replacer),
                                               np.random.choice(ix_neg,
                                                                size=len(ix_pos), 
                                                                replace=True)))

            if config.exp_bootstrap == "down":
                if len(ix_neg) > len(ix_pos):
                    ix_train = np.concatenate((np.random.choice(ix_pos
                                                                , size=len(ix_pos),
                                                                replace=replacer),
                                               np.random.choice(ix_neg,
                                                                size=len(ix_pos),
                                                                replace=False)))

                if len(ix_pos) > len(ix_neg):
                    ix_train = np.concatenate((np.random.choice(ix_neg,
                                                                size=len(ix_neg),
                                                                replace=replacer),
                                               np.random.choice(ix_pos,
                                                                size=len(ix_neg), 
                                                                replace=False)))

        results["ix_train"].append(ix_train)
        results["ix_test"].append(ix_test)

        dat = dict(train_x=X[ix_train, :],
                   test_x=X[ix_test,: ],
                   train_y=Y[ix_train],
                   test_y=Y[ix_test],
                   train_crisis_id=crisis_id[ix_train])

        # The error costs (false positve, false negative) determine how
        # the instances are weighted in the training set
        if isinstance(config.exp_error_costs, dict):
            class_costs = config.exp_error_costs
        elif config.exp_error_costs == "balanced": # objects are weighted,
            # such that the weighted proportion of objects
            #  contribute equally to the training set
            class_costs = {0: dat["train_y"].mean(), 1: 1 - dat["train_y"].mean()}
        elif config.exp_error_costs == "0.5":
            class_costs = {0: 0.5, 1: 0.5} # each object has the same weight.

        if config.exp_do_upsample: # upsample training set
            dat["train_x"], dat["train_y"], 
            group = upsample(dat["train_x"],
                             dat["train_y"],
                             group=dat["train_crisis_id"], costs=class_costs)
            class_costs_use = {0: 0.5, 1: 0.5}
            sample_weight = compute_sample_weight(class_costs_use, dat["train_y"])
            cv_hyper, cv_fold_vector = create_grouped_folds(dat['train_y'],
                                                          group, nfolds=hyper_folds,
                                                          reps=rep_cv)
        else: # create folds for the hyperparater search. (Nested cross-validation)
            group = dat["train_crisis_id"]
            class_costs_use = class_costs
            class_weight = weights_from_costs(class_costs_use, dat["train_y"])
            sample_weight = compute_sample_weight(class_weight, dat["train_y"])
            cv_hyper, cv_fold_vector = create_grouped_folds(dat['train_y'],
                                                          dat["train_crisis_id"],
                                                          nfolds=hyper_folds,
                                                          reps=rep_cv)

        # rescale all variables according to the training set
        scaler = StandardScaler()
        dat['train_x_scaled'] = scaler.fit_transform(dat['train_x'])
        dat['test_x_scaled'] = scaler.transform(dat['test_x'])

        results["data"].append(dat)

        python_algos = [x for x in algo_names if x[0:2] != "r_"]
        # Train and test PYTHON prediction models 
        data = {"trainx": dat['train_x_scaled'] ,
                "trainy": dat['train_y'],
                "testx": dat['test_x_scaled']
                }
        
        for algo in python_algos:
            out = globals()[algo](data,
                        config=config,
                        cv_hyper=cv_hyper,
                        group=group,
                        sample_weight=sample_weight,
                        do_cv = False, name = algo)
        
            append_results(results, out)


        # Some models are trained in R, we call the R script from here.
        # The R script loads the  training and test set as a csv,
        # We save them here and then save the results as a csv as well.
        r_algos = [x for x in algo_names if x[0:2] == "r_"]
        
        if len(r_algos) > 0:
            try:
                os.makedirs("r_data")
            except:
                pass

            train_r = pd.DataFrame(dat['train_y'],
                                   columns=["y"]).join(pd.DataFrame(dat['train_x']))
            test_r = pd.DataFrame(dat['test_y'],
                                  columns=["y"]).join(pd.DataFrame(dat['test_x']))
            ident = np.random.randint(100000000) # random name_suffix as a unique
            # identifier for the experiment
           
            train_r.to_csv('r_data/train_in' + str(ident) + '.csv', sep='\t')
            test_r.to_csv('r_data/test_in' + str(ident) + '.csv', sep='\t')
            pd.Series(cv_fold_vector).to_csv('r_data/cv_fold_vector' \
                     + str(ident) + '.csv', sep='\t', header = False)
               
    
            for r_algo in r_algos:
                out = rmodel(r_algo, class_costs_use, config, ident, 5)
                append_results(results, out)
                
            while True: 
                try:    
                    remove_file('r_data/out_to_python_' + str(r_algo) + str(ident) + '.csv') # remove R file
                    remove_file('r_data/train_in' + str(ident) + '.csv')
                    remove_file('r_data/test_in' + str(ident) + '.csv')
                    remove_file('r_data/cv_fold_vector' + str(ident) + '.csv')
                    break
                except:
                    pass
                
    return results

def append_results(results, add):

    """Appends the results obtained for a single fold to the previous results"""
    name = add["name"]
    ix_test = results["ix_test"][-1] # last element in list

    results['predictions'].loc[ix_test, name] = add["pred"]
    results['fits'][name].append(add["fit"])

    results["models"][name].append(add["model"])
    results["parameters"][name].append(add["hyper_params"])
    results["time"][name].append(add["time"])
    if not add["shapley"] is None:
        results["shapley"].loc[name, ix_test, :] = add["shapley"]

        if "shapley_inter" in add.keys():
            if not add["shapley_inter"] is None:
                results["shapley_inter"].loc[name, ix_test, :, :] = add["shapley_inter"]

In [ ]:
"""
"""



import xarray as xr
import subprocess
import pickle
import math
import warnings
import yaml
class Procedure:

    def __init__(self, config, df_in=None, file_name=None, name_suffix="", folder=None,
                 keep_models=False, keep_data=False, save_data=True, skipExperiment=False):

        """This is the Procedure class. Here the experiments are conducted and
        the results are saved to the hard drive.

        :param str Config: Config objects specifying data processing and the experimental setup
        :param pd.Data.Frame df_in: The input data. If 'None', the data is generated in this function
        :param str file_name: The name given to the output files. If 'None', the name is given by the Config._make_name
        :param str name_suffix: Characters that are appended to the file name. This parameter is useful when
            you want to add something to the automatically generated file names by Config._make_name.
        
        :param str folder: The folder where the results are saved. If 'None'
            the results are saved in the working directory
        
        :param Boolean keep_models: Whether the actual models should be saved in pickle files.
            This can require substantial disk space and is not recommended
        :param Boolean keep_data:  Whether all training and test set partitions should
            be saved in the pickle. This can also require substantial space and is not recommended
        :param Boolean save_data:  Whether the dataset on which the algorithms are trained
            and tested should be written to the hard drive

        :param Boolean skipExperiment: Whether the experiment should be skipped
            and only the existing pickle files should be aggregated.

         """
        self.collected = False # Indicates whether the results have been collected from the hard drive.
        self.name_suffix = name_suffix
        self.keep_models = keep_models
        self.keep_data = keep_data
        self.save_data = save_data
        self.config = config

        if file_name is None:
            self.file_name = config._make_name(self.name_suffix)
        else:
            self.file_name = file_name
        if folder is None:
            self.folder = "results/" + config._make_name(self.name_suffix) + "/"
        else:
            self.folder = folder
        try:
            os.makedirs(self.folder)
        except:
            pass

        with open(self.folder + 'config.yml', 'w') as outfile:
            yaml.dump(config, outfile, default_flow_style=False)

        # Create dataset
        if not df_in is None:
            print('Data set given with ')
            self.df = df_in.copy()
        else:
            self.df = create_data(self.config)
            print('Data set created with ')

        print('    ' + str(np.sum(self.df.crisis.values == 1)) + " Crises")
        print('    ' + str(np.sum(self.df.crisis.values == 0)) + " No crises")
        print('    ' + str(self.df.shape[1] - len(["year", "crisis",
                           "crisis_id", "iso"])) + " Features")



        self.metrics = ["auc", "accuracy", "balanced", "tp_rate", "fp_rate",
                        "tp", "tn", "fp", "fn"] # performance metrics
        self.results = list()
        if self.save_data:
            write_file(self.df, "data_" + self.file_name, # save the dataset
                      path=self.folder, shorten=False)
        self.X = self.df.copy()
        self.Y = self.X['crisis'].values.astype(int)
        self.X = self.X.drop(columns=['crisis', 'crisis_id', 'year', 'iso'])
        self.feature_names  = self.X.columns.values
        self.X = np.array(self.X)

        # run the experiment
        if not skipExperiment:
            self._do_experiment()
            self._save_pickle(keep_models=self.keep_models, keep_data=self.keep_data)
        # write the results to the hard drive
        self._write_results()
        # write the Shapley values to the hard drive
        if self.config.exp_do_shapley:
             for m in self.config.exp_algos:
                 # we cannot do Shapley decomposition for those models trained in R:
                if not m in ["r_elnet", "r_cart", "r_c50"]:
                    self._write_shapley(model_name=m)

    def _do_experiment(self):
        """Conduct the experiment"""
        self.results.append(train_and_test(self.df, self.config))
        self.config.exp_nrep = len(self.results)


    def _collect_results(self):
        """Read the results from the pickles of the individual iterations saved on the hard drive
        and adds results to self.results"""
        if not self.collected:
            self.results = []
            self.collected = True
            if os.path.exists(self.folder):
                file_list = os.listdir(self.folder)
                pickle_names = [s for s in file_list if "pickle_" in s]
                pickle_names = [s for s in pickle_names if self.file_name in s]

                for p in pickle_names:
                    o_old = pickle.load(open(self.folder + p, 'rb'))
                    self._add_results(o_old)
                    
        if not all_same([i["predictions"].shape for i in self.results]):
            raise ValueError("You try to merge results of different experiments. This is not possible.")
        
        if not all_same([set(i["predictions"].columns) for i in self.results]): # results of different sizes are merged
                    raise ValueError("You try to merge results of different models. This is not possible.")
                
    

    def _add_results(self, old_object):
        # the current results are added to a new object!
        if self.file_name != old_object.file_name:
            print("Experimental configurations do not match")
            return None

        self.results = self.results + old_object.results

    def _write_results(self, ix=None):
        """Write the results to the hard drive. This function collects the results from
        previous iterations (saved in the pickle files) of this experiment (by calling _collect_results)
        and processes them  to produce the csv files.
        
        :param boolean list ix: Used to subset the observations such that the results
            are saved and performance metrics are computed only for these observations.
            If it is 'None' all observations are used. """


        if ix is None: # ix is used
            ix_select = [True] * self.df.shape[0] # select all observations
        else:
            ix_names = self.df[["year", "iso"]].apply(lambda x: str(x[0]) + "_" + str(x[1]), axis=1).values
            ix_select = [x in ix for x in ix_names.tolist()]

        self._collect_results()

        out_pred = list()
        for i in range(len(self.results)):
            dout = self.results[i]["predictions"].copy()
            dout["crisis"] = self.df.crisis
            dout["year"] = self.df.year
            dout["iso"] = self.df.iso
            dout["iter"] = i

            # identify fold
            folds = [np.where(np.isin(np.arange(len(self.df.crisis)), 
                                      self.results[i]["ix_test"][j]), j + 1, 0) for j in
                     np.arange(len(self.results[i]["ix_test"]))]

            folds = np.vstack(folds).sum(0)

            dout["fold"] = folds
            out_pred.append(dout)
                
        pred_all = pd.concat(out_pred)
        write_file(pred_all , "all_pred" + "_" + self.file_name,
                  path=self.folder, shorten=False)

        # Three types of processing the results of the repeated cross-validation experiment:

        # ---- 1 --- #
        # Then performance metrics are computed for each iteration across all objects. Then the performance
        # metrics are averaged across iterations.
        output_metric_across = xr.DataArray(np.zeros((len(self.results), len(self.config.exp_algos),
                                                           len(self.metrics))) * float("nan"),
                                                 [('iterations', np.arange(len(self.results))),
                                                  ("algorithms", self.config.exp_algos),
                                                  ("metrics", self.metrics)])
        for r in np.arange(len(self.results)):
            res_across_folds = self.results[r]['predictions'].apply(lambda x:
                                                    np.array([performance_results(self.df.crisis.values[ix_select],
                                                    x[ix_select])[z] for z in self.metrics])).T
            res_across_folds.columns = self.metrics
            output_metric_across.loc[r, :, :] = res_across_folds

        # mean
        performance_across_mean = output_metric_across.mean(axis=0).to_pandas()

        # standard error
        performance_across_se = output_metric_across.std(axis=0).to_pandas() \
        / math.sqrt(float(len(self.results)))

        # add the iteration index to the output
        iters = [len(self.results)] * performance_across_se.shape[0]
        iters = pd.DataFrame({"iter": iters}, index=performance_across_se.index.values)

        # --- 2 --- #
        # The mean predicted value of each object is computed across all replications of the cross-validation.
        # Then the performance metrics are calculated based on the mean predicted values.
        pred_all_mean = [np.array(out_pred[x][self.config.exp_algos]) for x in range(len(out_pred))]
        pred_all_mean = pd.DataFrame(np.stack(pred_all_mean).mean(0))
        pred_all_mean.columns = self.config.exp_algos

        output_metric_append = pred_all_mean.apply(lambda x:
                            np.array([performance_results(self.df.crisis.values[ix_select], x[ix_select])[z]
                                                  for z in self.metrics])).T
        output_metric_append.columns = self.metrics

        # --- 3 --- #
        # The performance metrics are computed for each fold in each replication and are then averaged across
        # folds and replications.

        # This approach is only sensible for the cross-validation experiment. If we split training and test set by year,
        # this approach is equivalent to the first type of processing the results
        if self.config.exp_year_split is None:
            output_metric_fold = xr.DataArray(np.zeros((len(self.results), self.config.exp_nfolds,
                                                        len(self.config.exp_algos), len(self.metrics))) * float("nan"),
                                                     [('iterations', np.arange(len(self.results))),
                                                      ('folds', np.arange(self.config.exp_nfolds)),
                                                      ("algorithms", self.config.exp_algos),
                                                      ("metrics", self.metrics)])
            for r in np.arange(len(self.results)):
                for f in np.arange(self.config.exp_nfolds):
                    ix_fold = self.results[r]['ix_test'][f] # get objects of the folds
                    ix_fold = set(ix_fold).intersection(set(np.where(np.array(ix_select))[0])) # only investigate those that are on our ix_select
                    ix_fold = np.array(list(ix_fold))
                    res_in_fold = self.results[r]['predictions'].iloc[ix_fold, :].apply(lambda x:
                                    np.array([performance_results(self.df.crisis.values[ix_fold],
                                                                  x)[z] for z in self.metrics])).T
                    res_in_fold.columns = self.metrics
                    output_metric_fold.loc[r, f, :, :] = res_in_fold


            # First average across folds in single iteration, then average (and estimate SE) over the replications.
            output_metric_fold_mean_folds = output_metric_fold.mean(axis=1)
            output_metric_fold_mean = output_metric_fold_mean_folds.mean(axis=0).to_pandas()
            output_metric_fold_se = output_metric_fold_mean_folds.std(axis=0).to_pandas()/\
                math.sqrt(float(len(self.results)))

        # ix controls the subset selection of year - country pairs
        if ix is None:

            write_file(pd.concat([performance_across_mean, iters], axis=1),
                      "mean_iter_" + self.file_name, path=self.folder)
            write_file(pd.concat([performance_across_se, iters], axis=1),
                      "se_iter_" + self.file_name, path=self.folder)
            write_file(pd.concat([output_metric_append, iters], axis=1),
                      "mean_append_" + self.file_name,
                      path=self.folder)
            if self.config.exp_year_split is None:
                write_file(pd.concat([output_metric_fold_mean, iters], axis=1),
                          "mean_fold_" + self.file_name,
                          path=self.folder)
                write_file(pd.concat([output_metric_fold_se, iters], axis=1),
                          "se_fold_" + self.file_name,
                          path=self.folder)
        else:
            write_file(pd.concat([performance_across_mean, iters], axis=1),
                      "mean_iter_ix_" + self.file_name,
                      path=self.folder)
            write_file(pd.concat([performance_across_se, iters], axis=1),
                      "se_iter_ix_" + self.file_name,
                      path=self.folder)
            write_file(pd.concat([output_metric_append, iters], axis=1),
                      "mean_append_ix_" + self.file_name,
                      path=self.folder)

        print("After " + str(len(self.results)) + " iterations:")
        print(performance_across_mean.round(3))

        for algo in self.config.exp_algos:
            self._write_hyper_param(algo)

    def _write_hyper_param(self, algo):
        """ Writes hyperparameters to the hard drive."""
        
        try:

            params = self.results[0]["parameters"][algo][0].keys()
            out = {}
            for p in params:
                out[p] = [[z[p] for z in x['parameters'][algo]] for x in self.results]
                listed = [x['parameters'][algo] for x in self.results]
                listed = [item for sublist in listed for item in sublist]
                out[p] = [x[p] for x in listed]
                write_file(pd.DataFrame(out), "hyper_" + algo + "_" + self.file_name,
                  path=self.folder, shorten=False)
        except:
            pass



    def _write_shapley(self, model_name=None, **kwargs):
        """ Collects the results of the Shapley experiments from the pickle files
        and writes them into csv files.
        :param str model_name: Name of the model for which the Shapley values should be collected
        """
        self._collect_results()
        do_shap = self.config.exp_do_shapley
        if not do_shap:
            print("No Shapley values found!")
            return None
        shap_values = [np.array(x["shapley"].loc[model_name, :, :]) for x in self.results]
        nrep = len(shap_values)

        shap_values_mean = np.nanmean(np.dstack(shap_values), axis=2) # mean Shapley values across all observations
        shap_values_append = np.concatenate(shap_values, axis=0) # Shapley values appended for all replications

        pred_matrix = [self.results[x]["predictions"][model_name] for x in np.arange(len(self.results))]
        # mean predicted value across all replications:
        pred_mean = pd.Series(pd.concat((pred_matrix), axis=1).mean(axis=1), name="pred").values  
        # predicted values appended for all replications:
        pred_append = np.concatenate(pred_matrix, axis=0).astype(float)  

        # Prepare a data set with mean Shapley values
        shap_out_mean = pd.DataFrame(shap_values_mean, columns=self.feature_names)
        shap_out_mean["pred"] = pred_mean
        shap_out_mean["year"] = self.df["year"]
        shap_out_mean["iso"] = self.df["iso"]
        shap_out_mean["crisis"] = self.Y
        write_file(shap_out_mean, file_name="shapley_mean_" + model_name \
                  + "_" + self.file_name, path=self.folder, shorten=False)

        # Prepare a data set with the Shapley values of all replications
        shap_out_append = pd.DataFrame(shap_values_append, columns=self.feature_names)
        shap_out_append["pred"] = pred_append
        shap_out_append["year"] = np.tile(self.df["year"], nrep)
        shap_out_append["iso"] = np.tile(self.df["iso"], nrep)
        shap_out_append["crisis"] = np.tile(self.Y, nrep)

        write_file(shap_out_append, file_name="shapley_append_" + model_name \
                  + "_" + self.file_name, path=self.folder,
                  shorten=False)

    
    def _save_pickle(self, file_name=None, keep_models=False, keep_data=False):

        """ Saves the results of the iteration of the experiment into a pickle file
        :param Boolean keep_models: Whether the actual models should be saved in pickle files.
            This can require substantial disk space and is not recommended.
        :param Boolean keep_data:  Whether all training and test set partitions should 
            be saved in the pickle. This can also require substantial space and is not recommended.
        """

        if file_name is None:
            file_name = "pickle_" + self.file_name + "_" + str(np.random.randint(100000000))

        if not keep_models:
            for i in np.arange(len(self.results)):
                self.results[i]["models"] = None
        if not keep_data:
            for i in np.arange(len(self.results)):
                self.results[i]["data"] = None

        pickle.dump(self, open(self.folder + file_name +".p", "wb"))